In [3]:
import numpy as np
import os
import time

import meshcat
import meshcat.geometry as g
import meshcat.transformations as tf

import manifpy as manif

In [36]:
#colord def
red = "0xc92302"
blue = "0x0902c9"
green = "0x02e634"
purple = "0x7802e6"

In [37]:
vis = meshcat.Visualizer()
# vis.open()
vis.url()
vis.jupyter_cell()



You can open the visualizer by visiting the following URL:
http://127.0.0.1:7003/static/


In [42]:
T1 = manif.SE3.Identity()
T2 = manif.SE3.Random()
vis["box_start"].set_object(g.Box(np.array([0.1, 0.2, 0.3])), g.MeshLambertMaterial(color=red, reflectivity=0.8))
vis["box_end"].set_object(g.Box(np.array([0.1, 0.2, 0.3])), g.MeshLambertMaterial(color=blue, reflectivity=0.8))
vis["box_noise"].set_object(g.Box(np.array([0.1, 0.2, 0.3])), g.MeshLambertMaterial(color=green, reflectivity=0.8))
vis["box_smooth"].set_object(g.Box(np.array([0.1, 0.2, 0.3])), g.MeshLambertMaterial(color=purple, reflectivity=0.8))



#setup
vis["box_start"].set_transform(T1.transform())
vis["box_end"].set_transform(T2.transform())
vis["box_noise"].set_transform(T1.transform())
vis["box_smooth"].set_transform(T1.transform())


In [87]:
alpha = 0.001
beta = 0.1
noise = 0.1
T_tmp = T1
T_smooth = T1
for i in range(int(1/alpha) + 1):
    lamb = i * alpha
    # print(lamb)
    # T_tmp = (T1.log() * (1 - lamb) + T2.log() * lamb).exp()
    T_tmp = T1 + (T2 - T1)* lamb + manif.SE3Tangent.Random() * np.random.rand()*noise
    # T_smooth = T_smooth + (T_tmp - T_smooth) * beta
    T_smooth = T_smooth + manif.SE3Tangent((T_tmp - T_smooth).coeffs() * beta * np.array([1.0, 1.0, 1.0, 1.0, 1.0, 1.0]))


    vis["box_noise"].set_transform(T_tmp.transform())
    vis["box_smooth"].set_transform(T_smooth.transform())
    time.sleep(0.001)